In [1]:
import tensorflow as tf 
import tensorflow_datasets as tfds
import numpy as np 

def get_mnist_iter(tr_bs=32, ts_bs=32):
    train_size = 60000
    test_size = 10000
    (tr, tr_y), (ts, ts_y) = tf.keras.datasets.mnist.load_data()
    tr, ts = tf.reshape(tr, [-1, 28, 28, 1]), tf.reshape(ts, [-1, 28, 28, 1])
    tr_y, ts_y = tf.reshape(tr_y, [-1, 1]), tf.reshape(ts_y, [-1, 1])
    tr_dataset = tf.data.Dataset.from_tensor_slices({'images':tr, 'labels':tr_y})
    tr_dataset = tr_dataset.shuffle(train_size).batch(tr_bs, drop_remainder=True)
    ts_dataset = tf.data.Dataset.from_tensor_slices({'images':ts, 'labels':ts_y})
    ts_dataset = ts_dataset.shuffle(test_size).batch(ts_bs, drop_remainder=True)
    return(tr_dataset.as_numpy_iterator(), ts_dataset.as_numpy_iterator())
pass 

def CNN():
    x = tf.keras.Input([28, 28, 1])
    e1 = tf.keras.layers.Conv2D(16, (3,3), padding="SAME", strides=(2,2), activation=tf.nn.relu)(x) #[14,14]
    e1 = tf.keras.layers.Conv2D(16, (3,3), padding="SAME", strides=(1,1), activation=tf.nn.relu)(e1) #[14,14]
    e1 = tf.keras.layers.Conv2D(16, (3,3), padding="SAME", strides=(1,1), activation=tf.nn.relu)(e1) #[14,14]
    e2 = tf.keras.layers.Conv2D(32, (3,3), padding="SAME", strides=(2,2), activation=tf.nn.relu)(e1) #[7,7]
    e2 = tf.keras.layers.Conv2D(32, (3,3), padding="SAME", strides=(1,1), activation=tf.nn.relu)(e2) #[7,7]
    e2 = tf.keras.layers.Conv2D(32, (3,3), padding="SAME", strides=(1,1), activation=tf.nn.relu)(e2) #[7,7]
    e3 = tf.keras.layers.Conv2D(64, (3,3), padding="SAME", strides=(2,2), activation=tf.nn.relu)(e2) #[4,4]
    e3 = tf.keras.layers.Conv2D(64, (3,3), padding="SAME", strides=(1,1), activation=tf.nn.relu)(e3) #[4,4]
    e3 = tf.keras.layers.Conv2D(64, (3,3), padding="SAME", strides=(1,1), activation=tf.nn.relu)(e3) #[4,4]
    
    fc = tf.keras.layers.Flatten()(e3)
    fc1 = tf.keras.layers.Dense(128, activation=tf.nn.relu)(fc)
    fc2 = tf.keras.layers.Dense(64, activation=tf.nn.relu)(fc1)
    fc3 = tf.keras.layers.Dense(32, activation=tf.nn.relu)(fc2)
    
    out = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(fc3)
    
    return tf.keras.Model(inputs=x, outputs=out)
pass 

2023-10-09 09:09:18.156716: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-09 09:09:18.194736: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
cnn = CNN()
tr_iter, ts_iter = get_mnist_iter()

In [10]:
fetcher = next(tr_iter)
target, label = fetcher['images'], fetcher['labels']
label = tf.reshape(label, [-1])
label.shape

TensorShape([32])